# Description
Calculate the proportion of checklists with at least one observation of a given species for each grid cell and year.

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta
import math
import matplotlib.pyplot as plt

In [2]:
os.getcwd()

'/Users/alvastrand/Documents/OU/Research/notebooks/calculate_arrival_date/archive'

In [3]:
path = '/Users/alvastrand/Documents/OU/Research/data/'
os.chdir(path)

In [4]:
os.getcwd()

'/Users/alvastrand/Documents/OU/Research/data'

In [5]:
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)

In [6]:
start_date = '0101'
end_date = '0731'
month = 'Apr'
year = '2020'
countries_states = 'US_states_east_Mississippi'

In [7]:
subdir = 'output/'
filename = "obligate_aerial_insectivores_ebird_species_codes.csv"

df = pd.read_csv(subdir + filename)

print(len(df))

19


In [8]:
df.head(2)

,scientific_name,common_name,species_code,category,taxon_order,order,family,report_as
0,Antrostomus arizonae,Mexican Whip-poor-will,souwpw1,species,3533,Caprimulgiformes,Caprimulgidae,NaN
1,Antrostomus carolinensis,Chuck-will's-widow,chwwid,species,3510,Caprimulgiformes,Caprimulgidae,NaN


In [9]:
def name_of_function(species, start_date, end_date, month, year, *args):
    
    subdir = 'eBird/ebd_output/'
    
    if args != ():
        
        countries_states = args[0]
        
        filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
        '_complete_zerofilled_grid_cells_rel' + month + '-' + year + '.csv'
        print(filename)
    
    df = pd.read_csv(subdir + filename)
    print('len(df):', len(df))
    
    df['species_observed_binary_values'] = np.where(df['species_observed'] == True, 1.0, 0.0)
    df['year'] = df['observation_date'].astype(str).str[:4]
    df['year'] = pd.to_numeric(df['year'])
    df['day'] = df['observation_date'].astype(str).str[5:]
    
    # Filter

    df = df[df['year'] != 2020]
    print('len(df):', len(df))
    
#     for start_year in range(2002, 2003):
    for start_year in range(2003, 2004):
        
        print('start_year:', start_year)
        
        # Filter

        df = df[df['year'] >= start_year]
        print('len(df):', len(df))

        df_cnt = df[['checklist_id', 'grid_cell', 'day', 'observation_date']].groupby(
            ['grid_cell', 'day', 'observation_date']).count()
        df_cnt = df_cnt.rename(columns={'checklist_id': 'nb_checklists'})
        df_cnt = df_cnt.reset_index()
        print('len(df_cnt):', len(df_cnt))
        print("len(df_cnt['grid_cell'].unique()):", len(df_cnt['grid_cell'].unique()))

        start_month = 1
        start_day = 1

        end_month = 7
        end_day = 31

        start_date_dt = date(2020, start_month, start_day)
        end_date_dt = date(2020, end_month, end_day)

        list_dates = [start_date_dt + timedelta(days=i) for i in range((end_date_dt - start_date_dt).days + 1)]
        assert(len(list_dates) == 213)

        list_days = [i.strftime('%m-%d') for i in list_dates]
        assert(len(list_days) == 213)

        list_dates = [str(i) + '-' + j for j in list_days for i in range(start_year, 2020)]
        assert(len(list_dates) == len(range(start_year, 2020)) * 213)

        list_dates_repeated = list_dates * len(df_cnt['grid_cell'].unique())
        assert(len(list_dates_repeated) == len(df_cnt['grid_cell'].unique()) * len(list_dates))

        list_grid_cells_repeated = list(df_cnt['grid_cell'].unique()) * len(list_dates)
        list_grid_cells_repeated = sorted(list_grid_cells_repeated)
        assert(len(list_grid_cells_repeated) == len(list_dates_repeated))

        df_grid_cells_dates = pd.DataFrame({'grid_cell': list_grid_cells_repeated, 
                                            'observation_date': list_dates_repeated})
        df_grid_cells_dates['day'] = df_grid_cells_dates['observation_date'].astype(str).str[5:]
        df_grid_cells_dates = df_grid_cells_dates[['grid_cell', 'day', 'observation_date']]
        assert(len(df_grid_cells_dates) == len(list_grid_cells_repeated))

        df_cnt_grid_cells_dates = df_grid_cells_dates.merge(
            df_cnt, how='left', on=['grid_cell', 'day', 'observation_date'])
        df_cnt_grid_cells_dates['nb_checklists'] = df_cnt_grid_cells_dates['nb_checklists'].fillna(0)
        assert(len(df_cnt_grid_cells_dates) == len(df_grid_cells_dates))

        df_min_cnt_grid_cells_dates = df_cnt_grid_cells_dates[['grid_cell', 'day', 'nb_checklists']].groupby(
            ['grid_cell', 'day']).min()
        df_min_cnt_grid_cells_dates = df_min_cnt_grid_cells_dates.rename(
            columns={'nb_checklists': 'min_nb_checklists'})
        df_min_cnt_grid_cells_dates = df_min_cnt_grid_cells_dates.reset_index()
        df_min_cnt_grid_cells_dates['min_nb_checklists'] = \
        df_min_cnt_grid_cells_dates['min_nb_checklists'].astype(int)
        assert(len(df_min_cnt_grid_cells_dates) == len(df_cnt['grid_cell'].unique()) * len(list_days))

        df_cnt_zero = df_min_cnt_grid_cells_dates[['grid_cell', 'min_nb_checklists']].groupby(
            ['grid_cell']).agg(lambda x: x.eq(0).sum())
        df_cnt_zero = df_cnt_zero.rename(columns={'min_nb_checklists': 'cnt_zero'})
        df_cnt_zero = df_cnt_zero.reset_index()
        assert(len(df_cnt_zero) == len(df_cnt['grid_cell'].unique()))
        
        threshold = 212/2
        
        # Filter
        
        list_grid_cells = list(df_cnt_zero[df_cnt_zero['cnt_zero'] < threshold]['grid_cell'])
        print('len(list_grid_cells):', len(list_grid_cells))
#         print(len(list_grid_cells)/len(df_cnt['grid_cell'].unique()))
        
        df_dates = pd.DataFrame({'observation_date': list_dates})
        df_dates['day'] = df_dates['observation_date'].astype(str).str[5:]
        df_dates = df_dates[['day', 'observation_date']]
        assert(len(df_dates) == len(range(start_year, 2020)) * 213)
        
        df_sampled = pd.DataFrame()
        
#         for i in range(len(list_grid_cells)):

        i = 0
            
        print('i =', i)

        grid_cell_threshold = list_grid_cells[i]

        df_grid_cell = df[df['grid_cell'] == grid_cell_threshold]
        print('len(df_grid_cell):', len(df_grid_cell))

        df_min_cnt_grid_cell = df_min_cnt_grid_cells_dates[
            df_min_cnt_grid_cells_dates['grid_cell'] == grid_cell_threshold]
        df_min_cnt_grid_cell = df_min_cnt_grid_cell.drop(columns=['grid_cell'])
        assert(len(df_min_cnt_grid_cell) == 213)

        df_min_cnt_greater_than_grid_cell = df_min_cnt_grid_cell[df_min_cnt_grid_cell['min_nb_checklists'] > 0]

        len_df_min_cnt_greater_than_grid_cell = len(df_min_cnt_greater_than_grid_cell)

        print('len(df_min_cnt_greater_than_grid_cell) =', len_df_min_cnt_greater_than_grid_cell)

        df_min_cnt_greater_than_grid_cell = df_dates.merge(df_min_cnt_greater_than_grid_cell, on=['day'])
        assert(len(df_min_cnt_greater_than_grid_cell) == len_df_min_cnt_greater_than_grid_cell * 
               len(range(start_year, 2020)))

        list_dates_grid_cell = list(df_min_cnt_greater_than_grid_cell['observation_date'])
        list_min_cnt_grid_cell = list(df_min_cnt_greater_than_grid_cell['min_nb_checklists'])
            
    return df, df_cnt, df_cnt_grid_cells_dates, df_min_cnt_grid_cells_dates, df_cnt_zero, df_dates, \
df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell
            
#             for j in range(len(list_dates_grid_cell)):

#                 df_grid_cell_date = df_grid_cell[df_grid_cell['observation_date'] == list_dates_grid_cell[j]]
#                 df_grid_cell_date = df_grid_cell_date.sample(list_min_cnt_grid_cell[j], random_state=1)
#                 df_sampled = df_sampled.append(df_grid_cell_date)
            
#         print('len(df_sampled):', len(df_sampled))
                
#         # Calculate the number of checklists for each grid cell and date
#         df_cnt_sampled = df_sampled[['checklist_id', 'grid_cell', 'observation_date']].groupby(
#             ['grid_cell', 'observation_date']).count()
#         df_cnt_sampled = df_cnt_sampled.rename(columns={'checklist_id': 'nb_checklists'})
#         print(len(df_cnt_sampled))

#         # Calculate the number of checklists with at least one observation of the species of interest for each grid 
#         # cell and date
#         df_sum_sampled = df_sampled[['species_observed_binary_values', 'grid_cell', 'observation_date']].groupby([
#             'grid_cell', 'observation_date']).sum()
#         df_sum_sampled = df_sum_sampled.rename(columns={'species_observed_binary_values': 'nb_checklists_species'})
#         assert(len(df_sum_sampled) == len(df_cnt_sampled))

#         df_grp_sampled = df_cnt_sampled.merge(df_sum_sampled, left_index=True, right_index=True)
#         # Calculate the proportion of checklists with at least one observation of the species of interest for each 
#         # grid cell and date
#         df_grp_sampled['prop'] = df_grp_sampled['nb_checklists_species']/df_grp_sampled['nb_checklists']
#         df_grp_sampled = df_grp_sampled.reset_index()
#     #     df_grp_sampled['observation_date_dt'] = pd.to_datetime(df_grp_sampled['observation_date'])
#         df_grp_sampled['year'] = df_grp_sampled['observation_date'].astype(str).str[:4]
#         assert(len(df_grp_sampled) == len(df_cnt_sampled))

#         df_grid_cells_threshold = df[df['grid_cell'].isin(list_grid_cells)]
#         print('len(df_grid_cells_threshold):', len(df_grid_cells_threshold))

#         # Calculate the number of checklists for each grid cell and date
#         df_cnt_not_sampled = df_grid_cells_threshold[['checklist_id', 'grid_cell', 'observation_date']].groupby(
#             ['grid_cell', 'observation_date']).count()
#         df_cnt_not_sampled = df_cnt_not_sampled.rename(columns={'checklist_id': 'nb_checklists'})
#         print(len(df_cnt_not_sampled))

#         # Calculate the number of checklists with at least one observation of the species of interest for each grid 
#         # cell and date
#         df_sum_not_sampled = df_grid_cells_threshold[
#             ['species_observed_binary_values', 'grid_cell', 'observation_date']].groupby(
#             ['grid_cell', 'observation_date']).sum()
#         df_sum_not_sampled = df_sum_not_sampled.rename(
#             columns={'species_observed_binary_values': 'nb_checklists_species'})
#         assert(len(df_sum_not_sampled) == len(df_cnt_not_sampled))

#         df_grp_not_sampled = df_cnt_not_sampled.merge(df_sum_not_sampled, left_index=True, right_index=True)
#         # Calculate the proportion of checklists with at least one observation of the species of interest for each 
#         # grid cell and date
#         df_grp_not_sampled['prop'] = df_grp_not_sampled['nb_checklists_species']/df_grp_not_sampled['nb_checklists']
#         df_grp_not_sampled = df_grp_not_sampled.reset_index()
#         df_grp_not_sampled['year'] = df_grp_not_sampled['observation_date'].astype(str).str[:4]
#         assert(len(df_grp_not_sampled) == len(df_cnt_not_sampled))
        
#         df_t_days = pd.DataFrame({'t': list(range(1, 214)), 'day': list_days})
#         assert(len(df_t_days) == 213)

#         df_t_days_dates_years = df_t_days.merge(df_dates, on=['day'])
#         df_t_days_dates_years['year'] = df_t_days_dates_years['observation_date'].astype(str).str[:4]
#         assert(len(df_t_days_dates_years) == len(range(start_year, 2020)) * 213)

#         list_years_repeated = list(df_t_days_dates_years['year']) * len(list_grid_cells)
#         assert(len(list_years_repeated) == len(df_t_days_dates_years) * len(list_grid_cells))

#         list_t_repeated = list(df_t_days_dates_years['t']) * len(list_grid_cells)
#         assert(len(list_t_repeated) == len(df_t_days_dates_years) * len(list_grid_cells))

#         list_dates_threshold_repeated = list(df_t_days_dates_years['observation_date']) * len(list_grid_cells)
#         assert(len(list_dates_threshold_repeated) == len(df_t_days_dates_years) * len(list_grid_cells))

#         list_grid_cells_threshold_repeated = list_grid_cells * len(df_t_days_dates_years)
#         list_grid_cells_threshold_repeated = sorted(list_grid_cells_threshold_repeated)
#         assert(len(list_grid_cells_threshold_repeated) == len(df_t_days_dates_years) * len(list_grid_cells))
        
#         df_t_dates_years = pd.DataFrame({'grid_cell': list_grid_cells_threshold_repeated, 
#                                          'year': list_years_repeated, 't': list_t_repeated, 
#                                          'observation_date': list_dates_threshold_repeated})
#         assert(len(df_t_dates_years) == len(list_grid_cells) * len(range(start_year, 2020)) * 213)
        
#         df_grp_sampled_dates = df_t_dates_years.merge(
#             df_grp_sampled, how='left', on=['grid_cell', 'observation_date', 'year'])
#         df_grp_sampled_dates = df_grp_sampled_dates[
#             ['grid_cell', 'year', 't', 'observation_date', 'nb_checklists', 'nb_checklists_species', 'prop']]
#         df_grp_sampled_dates = df_grp_sampled_dates.sort_values(by=['grid_cell', 'year', 't'])
        
#         # Perform arcsine transformation on the proportions
#         df_grp_sampled_dates['prop_arcsine'] = df_grp_sampled_dates['prop'].apply(lambda x: np.arcsin(math.sqrt(x)))
#         # ?
        
#         assert(len(df_grp_sampled_dates) == len(list_grid_cells) * len(range(start_year, 2020)) * 213)
        
#         df_grp_not_sampled_dates = df_t_dates_years.merge(
#             df_grp_not_sampled, how='left', on=['grid_cell', 'observation_date', 'year'])
#         df_grp_not_sampled_dates = df_grp_not_sampled_dates[
#             ['grid_cell', 'year', 't', 'observation_date', 'nb_checklists', 'nb_checklists_species', 'prop']]
#         df_grp_not_sampled_dates = df_grp_not_sampled_dates.sort_values(by=['grid_cell', 'year', 't'])
        
#         # Perform arcsine transformation on the proportions
#         df_grp_not_sampled_dates['prop_arcsine'] = df_grp_not_sampled_dates['prop'].apply(
#             lambda x: np.arcsin(math.sqrt(x)))
#         # ?
        
#         assert(len(df_grp_not_sampled_dates) == len(list_grid_cells) * len(range(start_year, 2020)) * 213)
        
#         df_sum_prop_sampled = df_grp_sampled_dates[['grid_cell', 'year', 'prop']].groupby(
#             ['grid_cell', 'year']).sum()
#         df_sum_prop_sampled = df_sum_prop_sampled.reset_index()
#         df_sum_prop_sampled = df_sum_prop_sampled.rename(columns={'prop': 'sum_prop'})
#         assert(len(df_sum_prop_sampled) == len(list_grid_cells) * len(range(start_year, 2020)))
        
#         df_sum_prop_not_sampled = df_grp_not_sampled_dates[['grid_cell', 'year', 'prop']].groupby(
#             ['grid_cell', 'year']).sum()
#         df_sum_prop_not_sampled = df_sum_prop_not_sampled.reset_index()
#         df_sum_prop_not_sampled = df_sum_prop_not_sampled.rename(columns={'prop': 'sum_prop'})
#         assert(len(df_sum_prop_not_sampled) == len(list_grid_cells) * len(range(start_year, 2020)))
        
#         assert(len(df_sum_prop_sampled[df_sum_prop_sampled['sum_prop'] == 0]) == 0)
#         assert(len(df_sum_prop_not_sampled[df_sum_prop_not_sampled['sum_prop'] == 0]) == 0)
    
#         if args != ():
            
#             end_year = 2019

#             countries_states = args[0]

#             filename_sampled = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
#             '_complete_zerofilled_grid_cells_proportions_' + str(start_year) + '_' + str(end_year) + \
#             '_sampled_rel' + month + '-' + year + '.csv'
#             print(filename_sampled)
            
#             filename_not_sampled = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
#             '_complete_zerofilled_grid_cells_proportions_' + str(start_year) + '_' + str(end_year) + \
#             '_not_sampled_rel' + month + '-' + year + '.csv'
#             print(filename_sampled)

#         df_grp_sampled_dates.to_csv(subdir + filename_sampled, index=False)
#         df_grp_not_sampled_dates.to_csv(subdir + filename_not_sampled, index=False)

#     return df, df_cnt, df_cnt_grid_cells_dates, df_min_cnt_grid_cells_dates, df_cnt_zero, df_dates, \
# df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell, df_grp_sampled, df_grp_not_sampled, df_t_days, \
# df_t_days_dates_years, df_t_dates_years, df_grp_sampled_dates, df_grp_not_sampled_dates, df_sum_prop_sampled, \
# df_sum_prop_not_sampled

In [10]:
species = 'treswa'

df_grid_cells, df_cnt, df_cnt_grid_cells_dates, df_min_cnt_grid_cells_dates, df_cnt_zero, df_dates, \
df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell = name_of_function(
    species, start_date, end_date, month, year, countries_states)

# df_grid_cells, df_cnt, df_cnt_grid_cells_dates, df_min_cnt_grid_cells_dates, df_cnt_zero, df_dates, \
# df_min_cnt_grid_cell, df_min_cnt_greater_than_grid_cell, df_grp_sampled, df_grp_not_sampled, df_t_days, \
# df_t_days_dates_years, df_t_dates_years, df_grp_sampled_dates, df_grp_not_sampled_dates, df_sum_prop_sampled, \
# df_sum_prop_not_sampled = name_of_function(species, start_date, end_date, month, year, countries_states)

ebd_US_states_east_Mississippi_treswa_0101_0731_complete_zerofilled_grid_cells_relApr-2020.csv


/Users/alvastrand/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (11,29) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


len(df): 8351771
len(df): 7227884
start_year: 2003
len(df): 7148340
len(df_cnt): 226213
len(df_cnt['grid_cell'].unique()): 108
len(list_grid_cells): 6
i = 0
len(df_grid_cell): 149274
len(df_min_cnt_greater_than_grid_cell) = 174


In [11]:
df_grid_cells.head(2)

,checklist_id,last_edited_date,country,country_code,state,state_code,county,county_code,iba_code,bcr_code,usfws_code,atlas_block,locality,locality_id,locality_type,latitude,longitude,observation_date,time_observations_started,observer_id,sampling_event_identifier,protocol_type,protocol_code,project_code,duration_minutes,effort_distance_km,effort_area_ha,number_observers,all_species_reported,group_identifier,trip_comments,scientific_name,observation_count,species_observed,grid_cell,species_observed_binary_values,year,day
1,S1018111,2013-09-23 08:41:34,United States,US,Ohio,US-OH,Portage,US-OH-133,NaN,13.0,NaN,NaN,Mogadore Res - Sunnybrook Rd,L140837,P,41.063328,-81.373825,2003-01-04,14:15:00,obs18443,S1018111,Traveling,P22,EBIRD,30,0.402,NaN,1,True,NaN,NaN,Tachycineta bicolor,0,False,131,0.0,2003,01-04
2,S1018363,2016-06-26 17:20:27,United States,US,Illinois,US-IL,Lake,US-IL-097,NaN,22.0,NaN,NaN,Home,L132381,P,42.311466,-88.191399,2003-01-05,08:30:00,obs19231,S1018363,Stationary,P21,EBIRD,300,NaN,NaN,1,True,NaN,"Snow fell last night, more today. A raw wet co...",Tachycineta bicolor,0,False,128,0.0,2003,01-05


In [12]:
df_cnt.head(2)

,grid_cell,day,observation_date,nb_checklists
0,4,01-26,2014-01-26,1
1,4,02-02,2013-02-02,1


In [13]:
df_cnt_grid_cells_dates.head(2)

,grid_cell,day,observation_date,nb_checklists
0,4,01-01,2003-01-01,0.0
1,4,01-01,2004-01-01,0.0


In [14]:
df_min_cnt_grid_cells_dates.head(2)

,grid_cell,day,min_nb_checklists
0,4,01-01,0
1,4,01-02,0


In [15]:
df_cnt_zero.head(2)

,grid_cell,cnt_zero
0,4,213
1,5,213


In [16]:
df_dates.head(2)

,day,observation_date
0,01-01,2003-01-01
1,01-01,2004-01-01


In [17]:
df_min_cnt_grid_cell.head(2)

,day,min_nb_checklists
6177,01-01,1
6178,01-02,0


In [18]:
df_min_cnt_greater_than_grid_cell.head(2)

,day,observation_date,min_nb_checklists
0,01-01,2003-01-01,1
1,01-01,2004-01-01,1


In [ ]:
df_grp_sampled.head(2)

In [ ]:
df_grp_not_sampled.head(2)

In [ ]:
df_t_days.head(2)

In [ ]:
df_t_days_dates_years.head(2)

In [ ]:
df_t_dates_years.head(2)

In [ ]:
df_grp_sampled_dates.head(2)

In [ ]:
df_grp_not_sampled_dates.head(2)

In [ ]:
df_sum_prop_sampled.head(2)

In [ ]:
df_sum_prop_not_sampled.head(2)

In [ ]:
# df_grp_sampled['prop'].hist()
# plt.show()

In [ ]:
# df_grp_not_sampled['prop'].hist()
# plt.show()

In [ ]:
species_cnt = 0

for i in range(len(df)):
    
    print(i)
  
    species = df['species_code'].iloc[i]
    print(species)
    
    if ((species == 'souwpw1') | (species == 'bucnig') | (species == 'compoo') | (species == 'whtswi') | 
        (species == "blkswi") | (species == 'barswa')):
        continue
    
    df_grid_cells, df_cnt, df_sum, df_prop_greater_than, df_prop_greater_than_cnt, df_nb_prop, \
    df_grp = name_of_function(species, start_date, end_date, month, year, countries_states)
    
    species_cnt += 1

In [ ]:
print(species_cnt)